In [1]:
# Load Biospytial modules and etc.
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
import pandas as pd
import matplotlib.pyplot as plt
## Use the ggplot style
plt.style.use('ggplot')

In [2]:
# My mac
#data = pd.read_csv("/RawDataCSV/plotsClimateData_11092017.csv")
# My Linux desktop
data = pd.read_csv("/RawDataCSV/idiv_share/plotsClimateData_11092017.csv")


/opt/conda/envs/biospytial/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.SppN.mean()

5.1824128104220382

In [4]:
import geopandas as gpd

In [5]:
from django.contrib.gis import geos
from shapely.geometry import Point

In [6]:
data['geometry'] = data.apply(lambda z: Point(z.LON, z.LAT), axis=1)
new_data = gpd.GeoDataFrame(data)

## Let´s reproject to Alberts or something with distance

In [7]:
new_data.crs = {'init':'epsg:4326'}

### Uncomment to reproject
proj string taken from: http://spatialreference.org/

In [8]:
#new_data =  new_data.to_crs("+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=37.5 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs ")

# Model Fitting Using a GLM 

The general model will have the form:
$$ Biomass(x,y) = \beta_1 AET + \beta_2 Age + Z(x,y) + \epsilon $$
Where:
$\beta_1$ and $\beta_2$ are model parameters, $Z(x,y)$ is the *Spatial Autocorrelation* process and $\epsilon \sim N(0,\sigma^2)$

In [9]:
##### OLD #######
len(data.lon)
#X = data[['AET','StandAge','lon','lat']]
#X = data[['SppN','lon','lat']]
X = data[['lon','lat']]
#Y = data['plotBiomass']
Y = data[['SppN']]
## First step in spatial autocorrelation
#Y = pd.DataFrame(np.zeros(len(Y)))
## Let´s take a small sample only for the spatial autocorrelation
#import numpy as np
#sample_size = 2000
#randindx = np.random.randint(0,X.shape[0],sample_size)
#nX = X.loc[randindx]
#nY = Y.loc[randindx]
nX = X
nY = Y



In [10]:
## Small function for systematically selecting the k-th element of the data.
#### Sughgestion use for now a small k i.e. 10
systematic_selection = lambda k : filter(lambda i : not(i % k) ,range(len(data))) 

In [12]:
idx = systematic_selection(50)
print(len(idx))
nX = X.loc[idx]
nY = Y.loc[idx]
new_data = data.loc[idx]

737


In [13]:
len(new_data)

737

In [22]:
# Import GPFlow
import GPflow as gf
k = gf.kernels.Matern12(2,ARD=False,active_dims = [0,1]) + gf.kernels.Bias(1)
#k = gf.kernels.Matern12(2, lengthscales=1, active_dims = [0,1] ) + gf.kernels.Bias(1)
l = gf.likelihoods.Poisson()
model = gf.gpmc.GPMC(nX.as_matrix(), nY.as_matrix().reshape(len(nY),1).astype(float), k, l)


In [24]:
#model = gf.gpr.GPR(nX.as_matrix(),nY.as_matrix().reshape(len(nY),1).astype(float),k)
## If priors
#model.kern.matern12.lengthscales.prior = gf.priors.Gaussian(25.0,3.0)
#model.kern.matern32.variance.prior = GPflow.priors.Gamma(1.,1.)
#model.kern.bias.variance.prior = gf.priors.Gamma(1.,1.)
## Optimize

%time model.optimize(maxiter=100) # start near MAP


CPU times: user 35.9 s, sys: 3.04 s, total: 39 s
Wall time: 14.9 s


      fun: array([ 1913.86902295])
 hess_inv: <740x740 LbfgsInvHessProduct with dtype=float64>
      jac: array([  6.44528368e+00,   3.66167605e+00,   2.37126916e+00,
         3.99084934e+00,   4.58043614e+00,   2.44289078e+00,
         3.35879111e+00,   2.38506185e+00,   3.80880764e+00,
         1.71325578e+00,   3.89539947e+00,   2.76303685e+00,
         3.02354141e+00,   3.37260714e+00,   2.61401844e+00,
         1.21994417e+00,   3.72547986e+00,   7.23979866e-01,
         1.20894006e+00,   2.39778410e+00,   1.25884622e+00,
         1.12012964e+00,   1.40526563e+00,   1.36770157e+00,
         3.79159785e+00,   2.58935825e+00,   3.41322737e+00,
         3.13107522e+00,   2.20581891e+00,   2.16769663e+00,
         1.62946618e+00,   2.09358530e+00,   2.11091892e+00,
         2.37531408e+00,   1.81464034e+00,  -1.70074688e-02,
         2.00651916e+00,   1.39050955e+00,   2.13008291e+00,
         2.08779971e+00,   1.59072557e+00,   8.85038673e-01,
         1.42096409e+00,   9.34597443e-0

In [25]:
model.kern

Name,values,prior,constraint
kern.bias.variance,[ 2.63571738],None,+ve
kern.matern12.lengthscales,[ 0.40613709],None,+ve
kern.matern12.variance,[ 28.11890802],None,+ve


In [26]:
samples = model.sample(50, verbose=True, epsilon=1, Lmax=100)

## Fitted parameters (From HEC)

In [ ]:
model.kern.lengthscales = 25.4846122373
model.kern.variance = 10.9742076021
model.likelihood.variance = 4.33463026664

In [ ]:
%time mm = k.compute_K_symm(X.as_matrix())

In [ ]:
import numpy as np
Nn = 500
dsc = data
predicted_x = np.linspace(min(dsc.lon),max(dsc.lon),Nn)
predicted_y = np.linspace(min(dsc.lat),max(dsc.lat),Nn)
Xx, Yy = np.meshgrid(predicted_x,predicted_y)
## Fake richness
fake_sp_rich = np.ones(len(Xx.ravel()))
predicted_coordinates = np.vstack([ Xx.ravel(), Yy.ravel()]).transpose()
#predicted_coordinates = np.vstack([section.SppN, section.newLon,section.newLat]).transpose()

In [ ]:
len(predicted_coordinates)

In [ ]:
#We will calculate everything with the new model and parameters
#model = gf.gpr.GPR(X.as_matrix(),Y.as_matrix().reshape(len(Y),1).astype(float),k)

In [ ]:
%time means,variances = model.predict_y(predicted_coordinates)

In [ ]:
-

# Predictions with +2std.Dev

In [ ]:
#Using k-partition = 7
import cartopy
plt.figure(figsize=(17,11))

proj = cartopy.crs.PlateCarree()
ax = plt.subplot(111, projection=proj)


ax = plt.axes(projection=proj)
#algo = new_data.plot(column='SppN',ax=ax,cmap=colormap,edgecolors='')


#ax.set_extent([-93, -70, 30, 50])
ax.set_extent([-125, -60, 20, 50])
#ax.set_extent([-95, -70, 25, 45])

#ax.add_feature(cartopy.feature.LAND)
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
ax.add_feature(cartopy.feature.LAKES, alpha=0.9)
ax.stock_img()
#ax.add_geometries(new_data.geometry,crs=cartopy.crs.PlateCarree())
#ax.add_feature(cartopy.feature.RIVERS)
mm = ax.pcolormesh(Xx,Yy,means.reshape(Nn,Nn) + (2* np.sqrt(variances).reshape(Nn,Nn)),transform=proj )
#cs = plt.contour(Xx,Yy,np.sqrt(variances).reshape(Nn,Nn),linewidths=2,cmap=plt.cm.Greys_r,linestyles='dotted')
cs = plt.contour(Xx,Yy,means.reshape(Nn,Nn) + (2 * np.sqrt(variances).reshape(Nn,Nn)),linewidths=2,colors='k',linestyles='dotted',levels=range(1,20))
plt.clabel(cs, fontsize=16,inline=True,fmt='%1.1f')
#ax.scatter(new_data.lon,new_data.lat,edgecolors='',color='white',alpha=0.6)
plt.colorbar(mm)
plt.title("Predicted Species Richness + 2stdev")

# Predicted means

In [ ]:
#Using k-partition = 7
import cartopy
plt.figure(figsize=(17,11))

proj = cartopy.crs.PlateCarree()
ax = plt.subplot(111, projection=proj)


ax = plt.axes(projection=proj)
#algo = new_data.plot(column='SppN',ax=ax,cmap=colormap,edgecolors='')


#ax.set_extent([-93, -70, 30, 50])
ax.set_extent([-125, -60, 20, 50])
#ax.set_extent([-95, -70, 25, 45])

#ax.add_feature(cartopy.feature.LAND)
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
ax.add_feature(cartopy.feature.LAKES, alpha=0.9)
ax.stock_img()
#ax.add_geometries(new_data.geometry,crs=cartopy.crs.PlateCarree())
#ax.add_feature(cartopy.feature.RIVERS)
mm = ax.pcolormesh(Xx,Yy,means.reshape(Nn,Nn),transform=proj )
#cs = plt.contour(Xx,Yy,np.sqrt(variances).reshape(Nn,Nn),linewidths=2,cmap=plt.cm.Greys_r,linestyles='dotted')
cs = plt.contour(Xx,Yy,means.reshape(Nn,Nn),linewidths=2,colors='k',linestyles='dotted',levels=range(1,20))
plt.clabel(cs, fontsize=16,inline=True,fmt='%1.1f')
#ax.scatter(new_data.lon,new_data.lat,edgecolors='',color='white',alpha=0.6)
plt.colorbar(mm)
plt.title("Predicted Species Richness")


In [ ]:
#Using k-partition = 7
import cartopy
plt.figure(figsize=(17,11))

proj = cartopy.crs.PlateCarree()
ax = plt.subplot(111, projection=proj)


ax = plt.axes(projection=proj)
#algo = new_data.plot(column='SppN',ax=ax,cmap=colormap,edgecolors='')


#ax.set_extent([-93, -70, 30, 50])
ax.set_extent([-125, -60, 20, 50])
#ax.set_extent([-95, -70, 25, 45])

#ax.add_feature(cartopy.feature.LAND)
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
ax.add_feature(cartopy.feature.LAKES, alpha=0.9)
ax.stock_img()
#ax.add_geometries(new_data.geometry,crs=cartopy.crs.PlateCarree())
#ax.add_feature(cartopy.feature.RIVERS)
mm = ax.pcolormesh(Xx,Yy,means.reshape(Nn,Nn) - (2* np.sqrt(variances).reshape(Nn,Nn)),transform=proj )
#cs = plt.contour(Xx,Yy,np.sqrt(variances).reshape(Nn,Nn),linewidths=2,cmap=plt.cm.Greys_r,linestyles='dotted')
cs = plt.contour(Xx,Yy,means.reshape(Nn,Nn) - (2 * np.sqrt(variances).reshape(Nn,Nn)),linewidths=2,colors='k',linestyles='dotted',levels=[4.0,5.0,6.0,7.0,8.0])
plt.clabel(cs, fontsize=16,inline=True,fmt='%1.1f')
#ax.scatter(new_data.lon,new_data.lat,edgecolors='',color='white',alpha=0.6)
plt.colorbar(mm)
plt.title("Predicted Species Richness - 2stdev")

## Model Analysis


In [ ]:
model.get_parameter_dict()

## Let's calculate the residuals

In [ ]:
X_ = data[['LON','LAT']]
%time Y_hat = model.predict_y(X_)

In [ ]:
pred_y = pd.DataFrame(Y_hat[0])
var_y = pd.DataFrame(Y_hat[1])

In [ ]:
new_data['pred_y'] = pred_y
new_data['var_y'] = var_y

In [ ]:
new_data= new_data.assign(error=lambda y : (y.SppN - y.pred_y)**2 )

In [ ]:
new_data.error.hist(bins=50)

In [ ]:
print(new_data.error.mean())
print(new_data.error.std())

## Experiment
In this section we will bring a Raster Data from the US, using Biospytial Raster API. 
1. First select a polygon, then get A Raster FRom there, say Mean Temperature.

In [ ]:
import raster_api.tools as rt
from raster_api.models import MeanTemperature,ETOPO1,Precipitation,SolarRadiation
from sketches.models import Country


In [ ]:
## Select US
us_border = Country.objects.filter(name__contains='United States')[1]

In [ ]:
from django.db import close_old_connections

In [ ]:
close_old_connections()

In [ ]:
#Get Raster API
us_meantemp = rt.RasterData(Precipitation,us_border.geom)
us_meantemp.getRaster()

In [ ]:
us_meantemp.display_field()

In [ ]:
%time coords = us_meantemp.getCoordinates()